In [18]:
import torch
from torch import Tensor
import numpy as np

EPSILON = np.finfo(np.float32).tiny

In [3]:
class SubsetOperator(torch.nn.Module):
    def __init__(self, k, tau=1.0, hard=False):            # k is the number of samples we want, tau is the temperature parameter and hard:denotes if we want hard or soft samples
        super(SubsetOperator, self).__init__()
        self.k = k
        self.hard = hard
        self.tau = tau

    def forward(self, scores):                                # scores take in weights of each sample      # scores: Typical shape: [batch_size,n] or [batch_size,n,1]
        m = torch.distributions.gumbel.Gumbel(torch.zeros_like(scores), torch.ones_like(scores))
        g = m.sample()
        scores = scores + g

        # continuous top k  (we can later modify this to also output S_WRS, we will just need each onehot_approx to be stored seperately - then it will give k soft vectors)
        khot = torch.zeros_like(scores)
        onehot_approx = torch.zeros_like(scores)
        for i in range(self.k):
            khot_mask = torch.max(1.0 - onehot_approx, torch.tensor([EPSILON]))            # we can autodiff through this, there is no issue .
            # khot_mask = torch.max(1.0 - onehot_approx, torch.tensor([EPSILON]).cuda())      #CHECK MIGHT NEED TO PUT DEVICE HERE,
            scores = scores + torch.log(khot_mask)
            onehot_approx = torch.nn.functional.softmax(scores / self.tau, dim=1)
            khot = khot + onehot_approx

        if self.hard:
            # will do straight through estimation if training
            khot_hard = torch.zeros_like(khot)
            val, ind = torch.topk(khot, self.k, dim=1)             #This line uses the torch.topk function to find the top self.k elements in each row (since dim=1) of the khot tensor.  val will store the values of these top elements, and ind will store their indices.
            khot_hard = khot_hard.scatter_(1, ind, 1)              #Here, the scatter_ function is used to take the zero tensor khot_hard and set the indices specified in ind to 1. This effectively creates a "hard" version of khot where only the top self.k elements in each row are set to 1, and the rest are 0. The underscore at the end of scatter_ indicates that this operation is done in-place, modifying khot_hard directly.
            res = khot_hard - khot.detach() + khot                 #This line is a bit trickier. It's part of a technique called the Straight-Through Estimator (STE). khot.detach() creates a tensor that does not require gradients, effectively a constant in terms of backpropagation.  By subtracting khot.detach() and then adding khot, you replace the gradients of khot_hard with those of khot during backpropagation. This is because khot_hard - khot.detach() stops the gradient from flowing through the hard assignment. The result is that during the forward pass, res acts like the hard assignment (since khot.detach() has no effect), but during the backward pass (gradient computation), it behaves like khot (since khot_hard - khot.detach() has no gradient).
        else:
            res = khot

        return res

In [23]:
scores = [[[1.0],[2.0],[3.0],[4.0]],[[2.0],[3.0],[4.0],[5.0]]]
#scores = [[1.0,2.0,3.0,4.0],[2.0,3.0,4.0,5.0]]
scores_tensor = torch.tensor(scores)
print(scores_tensor.shape)
m = torch.distributions.gumbel.Gumbel(torch.zeros_like(scores_tensor), torch.ones_like(scores_tensor))
g = m.sample()
print(g)
scores_tensor=scores_tensor+g

khot = torch.zeros_like(scores_tensor)
onehot_approx = torch.zeros_like(scores_tensor)
for i in range(2):
    khot_mask = torch.max(1.0 - onehot_approx, torch.tensor([EPSILON]))        # we can autodiff through this, there is no issue .
    scores_tensor = scores_tensor + torch.log(khot_mask)
    onehot_approx = torch.nn.functional.softmax(scores_tensor / 0.2, dim=1)
    khot = khot + onehot_approx

print(khot)